In [1]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mnist = keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
y_train[100]

5

In [5]:
x_train, x_test = x_train / 255, x_test / 255

In [6]:
x_train = x_train[..., tf.newaxis].astype("float32")

In [7]:
x_train.shape

(60000, 28, 28, 1)

In [8]:
x_test = x_test[..., tf.newaxis].astype("float32")

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

In [10]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [11]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [12]:
class MyMode(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(128, activation='relu')
        self.d2 = keras.layers.Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [22]:
model = MyMode()

In [23]:
loss_function = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()

In [24]:
train_loss_metric = keras.metrics.Mean(name='train_loss')
train_accuracy_metric = keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [25]:
test_loss_metric = keras.metrics.Mean(name='test_loss')
test_accuracy_metric = keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [29]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_function(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss_metric(loss)
    train_accuracy_metric(labels, predictions)

In [30]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_function(labels, predictions)
    
    test_loss_metric(t_loss)
    test_accuracy_metric(labels, predictions)

In [31]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss_metric.reset_state()
    train_accuracy_metric.reset_state()
    test_loss_metric.reset_state()
    test_accuracy_metric.reset_state()
    
    for images, labels in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    print(
        f'Epoch {epoch + 1}, '
        f"Loss: {train_loss_metric.result():0.2f}, "
        f'Accuracy: {train_accuracy_metric.result() * 100:0.2f}, '
        f"Test loss: {test_loss_metric.result():0.2f}, "
        f"Test Accuracy: {test_accuracy_metric.result() * 100:0.2f}"
    )

Epoch 1, Loss: 0.14, Accuracy: 95.87, Test loss: 0.07, Test Accuracy: 97.67
Epoch 2, Loss: 0.04, Accuracy: 98.69, Test loss: 0.05, Test Accuracy: 98.23
Epoch 3, Loss: 0.02, Accuracy: 99.30, Test loss: 0.05, Test Accuracy: 98.40
Epoch 4, Loss: 0.01, Accuracy: 99.59, Test loss: 0.06, Test Accuracy: 98.43
Epoch 5, Loss: 0.01, Accuracy: 99.63, Test loss: 0.06, Test Accuracy: 98.34


**Keras Subclassing API** là một cách tiếp cận để xây dựng mô hình trong Keras bằng cách kế thừa từ lớp `tf.keras.Model`. Phương pháp này cho phép bạn tạo ra các mô hình phức tạp hơn và linh hoạt hơn, đặc biệt khi cần tùy chỉnh cách thức forward pass, loss function hoặc cách thức huấn luyện.

### Lợi ích của Keras Subclassing API
1. **Tùy chỉnh linh hoạt**: Bạn có thể định nghĩa các lớp riêng, cho phép bạn tự do trong việc xây dựng kiến trúc mạng và cách thức xử lý dữ liệu.
2. **Kiểm soát tốt hơn**: Bạn có thể dễ dàng kiểm soát từng bước trong quá trình huấn luyện, từ việc tính toán loss đến cập nhật weights.
3. **Khả năng mở rộng**: Thích hợp cho các ứng dụng phức tạp, chẳng hạn như các mô hình có nhiều đầu ra hoặc nhiều loại loss khác nhau.

### Ví dụ sử dụng Keras Subclassing API
```python
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

model = MyModel()

# Định nghĩa optimizer và loss
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Custom training loop
for epoch in range(epochs):
    for x_batch, y_batch in dataset:
        with tf.GradientTape() as tape:
            logits = model(x_batch)
            loss_value = loss_fn(y_batch, logits)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
```

### Tại sao nó có thể hiệu quả hơn so với `model.fit()`?
- **Tùy chỉnh quá trình huấn luyện**: Với custom training loop, bạn có thể thêm các logic đặc biệt như điều chỉnh learning rate trong quá trình huấn luyện, áp dụng các hình thức regularization khác nhau, hoặc thực hiện việc kiểm tra điều kiện dừng (early stopping) mà không bị giới hạn bởi các phương thức tự động hóa của `model.fit()`.
- **Kiểm soát trực tiếp quá trình tính toán gradient**: Bạn có thể can thiệp vào cách thức gradients được tính toán và áp dụng, giúp tối ưu hóa quá trình huấn luyện cho các mô hình phức tạp hơn.

Tóm lại, Keras Subclassing API cho phép bạn xây dựng các mô hình phức tạp và tùy chỉnh cách huấn luyện, mang lại nhiều lợi ích cho các ứng dụng yêu cầu cao về hiệu suất và khả năng mở rộng.

`tf.GradientTape()` là một lớp trong TensorFlow được sử dụng để ghi lại các phép toán mà bạn thực hiện trên các tensors để tính toán gradients (đạo hàm) một cách tự động. Nó rất hữu ích trong quá trình huấn luyện mô hình, cho phép bạn dễ dàng tính toán gradient của một hàm mất mát (loss function) đối với các tham số (weights) của mô hình.

### Cách hoạt động của `tf.GradientTape()`
1. **Ghi lại phép toán**: Khi bạn sử dụng `tf.GradientTape()`, tất cả các phép toán mà bạn thực hiện trên tensors sẽ được ghi lại trong một ngữ cảnh. Điều này cho phép TensorFlow biết cách để tính toán gradient sau đó.
   
2. **Tính toán gradient**: Sau khi bạn đã thực hiện các phép toán và ghi lại chúng, bạn có thể gọi phương thức `gradient()` để tính toán gradient của một giá trị (như loss) theo một hoặc nhiều tensors.

### Cú pháp sử dụng
```python
import tensorflow as tf

# Giả sử bạn có một mô hình và một hàm mất mát
model = ...  # Khởi tạo mô hình
loss_fn = ...  # Hàm mất mát

# Khởi tạo GradientTape
with tf.GradientTape() as tape:
    predictions = model(inputs)  # Tính toán dự đoán
    loss_value = loss_fn(targets, predictions)  # Tính toán giá trị mất mát

# Tính toán gradients
grads = tape.gradient(loss_value, model.trainable_variables)

# Cập nhật weights
optimizer.apply_gradients(zip(grads, model.trainable_variables))
```

### Lợi ích của việc sử dụng `tf.GradientTape()`
- **Tính toán tự động**: Giúp bạn không phải tính toán gradient thủ công, giảm thiểu lỗi và tiết kiệm thời gian.
- **Tùy chỉnh**: Bạn có thể dễ dàng điều chỉnh cách mà gradients được tính toán, cho phép bạn áp dụng các kỹ thuật tối ưu hóa phức tạp hơn.
- **Hỗ trợ cho các mô hình phức tạp**: Làm cho việc xây dựng và huấn luyện các mô hình sâu và phức tạp trở nên dễ dàng hơn.

### Kết luận
`tf.GradientTape()` là một công cụ mạnh mẽ giúp bạn quản lý và tính toán gradients trong TensorFlow, đặc biệt hữu ích trong các quy trình huấn luyện mô hình machine learning.

Lý do dòng lệnh `grads = tape.gradient(loss_value, model.trainable_variables)` được thực hiện ngoài `with tf.GradientTape() as tape` là vì `tf.GradientTape` chỉ có nhiệm vụ ghi lại các phép toán để tính toán gradient, nhưng không thực hiện việc tính gradient ngay lập tức. Việc tính toán gradient chỉ xảy ra khi bạn gọi `tape.gradient(...)` sau khi các phép toán đã được ghi lại.

### Chi tiết
1. **Ghi lại các phép toán**: Trong khối `with tf.GradientTape() as tape`, TensorFlow sẽ ghi lại các phép toán được thực hiện trên tensors. Khi bạn tính toán `predictions` và `loss_value`, các phép toán này đều được ghi lại bởi `tape`.

2. **Tính gradient sau khi đã có phép toán**: Sau khi ra khỏi khối `with`, các phép toán đã được ghi lại trong `tape`. Chỉ khi gọi `tape.gradient(loss_value, model.trainable_variables)`, TensorFlow mới sử dụng những phép toán đã ghi lại này để tính gradient của `loss_value` với các biến huấn luyện (`trainable_variables`) của mô hình.

### Tại sao không tính gradient bên trong `with`?
Nếu bạn tính gradient bên trong `with tf.GradientTape()`, thì `tape` sẽ tiếp tục ghi lại các phép toán, điều này không cần thiết và có thể làm tăng sử dụng bộ nhớ. Thay vào đó, việc tính gradient thường được thực hiện ngay sau khi khối `with` kết thúc.

### Tóm lại
Việc thực hiện `tape.gradient` ngoài `with` giúp cho việc ghi và tính gradient tách biệt, tối ưu bộ nhớ và hiệu năng khi huấn luyện mô hình.